In [ ]:
!pip install sklearn_crfsuite
!pip install indic-nlp-library

In [21]:
import pycrfsuite

def extract_features(sentence, index,sentences):
    # Extract features for a given word in a sentence
    word = sentence[index]

    prev_word = ""
    if index == 0:
      prev_word = "BOS"
    elif sentences[0][index-1][0] == '.':
      prev_word = "BOS"
    else:
      prev_word = sentence[index-1]

    next_word = ""
    if index == len(sentences[0])-1:
      next_word = "EOS"
    elif sentences[0][index+1][0] == '.':
      next_word = "EOS"
    else:
      next_word = sentence[index+1]

    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'prev_word=' + prev_word,
        'next_word=' + next_word
    ]
    return features

# Extract labels for each word in a sentence
def extract_labels(sentence):
    return [label for _, label in sentence]

# Convert the dataset to CRF format
def convert_to_crf_format(sentences):
    crf_data = []
    for sentence in sentences:
        tokens = [token for token, _ in sentence]
        labels = extract_labels(sentence)
        features = [extract_features(tokens, i,sentences_1) for i in range(len(tokens))]
        crf_data.append((features, labels))
    return crf_data

import pandas as pd

# Read the Excel file into a DataFrame
# df = pd.read_excel('/content/drive/MyDrive/final_out_v3.xlsx', usecols=["token","tag"])
df = pd.read_excel('/content/final_out_v3.xlsx', usecols=["token","tag"])
df['token'] = df['token'].astype(str)

# Convert the DataFrame to a list of tuples
val = list(list(df.itertuples(index=False, name=None)))
sentences_1 = []
sentences_1.append(val)

# Convert dataset to CRF format
crf_data = convert_to_crf_format(sentences_1)

# Initialize the CRF trainer
trainer = pycrfsuite.Trainer()

# Add data to the trainer
for xseq, yseq in crf_data:
    # try:
    trainer.append(xseq, yseq)
    # except:
    #   print(xseq)

# Set trainer parameters
trainer.set_params({
    'c1': 0.01,  # coefficient for L1 regularization
    'c2': 0.001,  # coefficient for L2 regularization
    'max_iterations': 1000,  # maximum number of iterations
    'feature.possible_transitions': True  # include transitions that are possible, but not observed
})

# Train the CRF model
val = trainer.train('ner_tamil_model.crfsuite')

print("CRF model training complete.")


Streaming output truncated to the last 5000 lines.
***** Iteration #447 *****
Loss: 2072.961760
Feature norm: 518.117676
Error norm: 13.652328
Active features: 19231
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.090

***** Iteration #448 *****
Loss: 2072.884590
Feature norm: 518.098620
Error norm: 8.188729
Active features: 19232
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.094

***** Iteration #449 *****
Loss: 2072.859479
Feature norm: 518.116882
Error norm: 13.021902
Active features: 19227
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.090

***** Iteration #450 *****
Loss: 2072.777012
Feature norm: 518.098359
Error norm: 7.575590
Active features: 19235
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.095

***** Iteration #451 *****
Loss: 2072.747167
Feature norm: 518.120033
Error norm: 12.610812
Active features: 19231
Line

In [20]:
import pycrfsuite
from indicnlp.tokenize import indic_tokenize

def extract_features(sentence, index,sentences):
    # Extract features for a given word in a sentence
    word = sentence[index]

    # print()
    prev_word = ""
    if index == 0:
      prev_word = "BOS"
    elif sentences[0][index-1][0] == '.':
      prev_word = "BOS"
    else:
      prev_word = sentence[index-1]

    next_word = ""
    if index == len(sentence)-1:
      next_word = "EOS"
    elif sentences[0][index+1][0] == '.':
      next_word = "EOS"
    else:
      next_word = sentence[index+1]
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'prev_word=' + prev_word,
        'next_word=' + next_word
    ]
    return features

def predict_labels(sentence, model):
    # Predict labels for a sentence using the trained CRF model
    features = [extract_features(sentence, i,test_sentences) for i in range(len(sentence))]
    return model.tag(features)

def test_model(sentences, model):
    # Test the CRF model on the given sentences
    for sentence in sentences:
        tokens = [token for token in sentence]
        predicted_labels = predict_labels(tokens, model)
        output = []
        for i in range(len(tokens)):
          output.append((tokens[i],predicted_labels[i]))
        print(output)

# Load the trained CRF model
model = pycrfsuite.Tagger()
model.open('ner_tamil_model.crfsuite')

sent = '''
Brado Jewellery பெண்கள் மற்றும் சிறுமிகளுக்கான மைக்ரோ தங்க முலாம் பூசப்பட்ட வெள்ளை அமெரிக்க வைர சதுரம் மற்றும் ஒரு காதல் இதய வடிவ பதக்கம் காதல் இதய சங்கிலி காம்போ 3 நெக்லஸ் (மஞ்சள்) . விற்பனை தொகுப்பு - மொத்த சங்கிலி: 3, மொத்த பதக்கம்: 3, அளவு: 18 in முழு நீளம் பிராண்ட்: Brado Jewellery, நாங்கள் நகைகளை பரிசளிக்கக்கூடிய பிராண்ட் பெட்டியுடன் வழங்குகிறோம், சரியான பேக்கிங் - அதிக பாதுகாப்புக்கு, நிறுவனத்தின் டேக் இணைக்கப்பட்டது - ப்ரோஃபுக்கு நீங்கள் அசல் பிராண்ட் தயாரிப்பைப் பெறுகிறீர்கள்; பயன்பாடு: தினசரி உடைகள், திருவிழா, திருமணம் மற்றும் நிகழ்ச்சிகள் சரியான பரிசு: உங்கள் பிரியமானவர்களுக்கு பொருத்தமான காதலர் தின, பிறந்தநாள், திருமண நாள் பரிசு. பெண்கள் நகைகளை விரும்புகிறார்கள், குறிப்பாக பாரம்பரிய நகைகள் பெண்களுக்கு மிகவும் பிடித்தமானவை. அவர்கள் அதைப் பல்வேறு தருணங்கள், திருமணம் மற்றும் பண்டிகை காலங்களில் அணிகிறார்கள். அவர்கள் தினசரி உபயோகத்திற்காகவும் இதை அணியலாம் உயர் தரம் மற்றும் சருமத்திற்கு உகந்தது: சர்வதேச மற்றும் இந்திய அரசாங்கத் தரத்தின்படி உயர் தரம் சருமத்திற்கு மிகவும் உகந்ததாக இருக்கிறது. இது ஒவ்வாமை எதிர்ப்பு மற்றும் சருமத்திற்கு பாதுகாப்பான நச்சு இல்லாத பொருட்களிலிருந்து தயாரிக்கப்படுகிறது. வலி மற்றும் வீக்கம் எதுவும் ஏற்படாமல் நீண்ட நேரம் அணிந்தாலும் ஆறுதலாக இருக்கும். பிரீமியம் தரமான மெட்டீரியல் கொண்டு தயாரிக்கப்படுகிறது, பல ஆண்டுகளுக்குப் பிறகும் புதியது போலவே தோற்றமளிக்கும். துறை பெயர்: பெண்கள்
'''

tokenized_sentences = [indic_tokenize.trivial_tokenize(sent.strip())]
test_sentences = tokenized_sentences

# Test the model
test_model(test_sentences, model)


[('Brado', 'O'), ('Jewellery', 'O'), ('பெண்கள்', 'GENDER'), ('மற்றும்', 'O'), ('சிறுமிகளுக்கான', 'PERSON'), ('மைக்ரோ', 'O'), ('தங்க', 'O'), ('முலாம்', 'O'), ('பூசப்பட்ட', 'O'), ('வெள்ளை', 'O'), ('அமெரிக்க', 'O'), ('வைர', 'O'), ('சதுரம்', 'O'), ('மற்றும்', 'O'), ('ஒரு', 'O'), ('காதல்', 'O'), ('இதய', 'O'), ('வடிவ', 'O'), ('பதக்கம்', 'O'), ('காதல்', 'O'), ('இதய', 'O'), ('சங்கிலி', 'O'), ('காம்போ', 'O'), ('3', 'O'), ('நெக்லஸ்', 'O'), ('(', 'PUN'), ('மஞ்சள்', 'COLOR'), (')', 'PUN'), ('.', 'PUN'), ('விற்பனை', 'O'), ('தொகுப்பு', 'O'), ('-', 'PUN'), ('மொத்த', 'O'), ('சங்கிலி', 'O'), (':', 'PUN'), ('3', 'O'), (',', 'PUN'), ('மொத்த', 'O'), ('பதக்கம்', 'O'), (':', 'PUN'), ('3', 'O'), (',', 'PUN'), ('அளவு', 'O'), (':', 'PUN'), ('18', 'SIZE'), ('in', 'SIZE'), ('முழு', 'O'), ('நீளம்', 'O'), ('பிராண்ட்', 'O'), (':', 'PUN'), ('Brado', 'O'), ('Jewellery', 'O'), (',', 'PUN'), ('நாங்கள்', 'O'), ('நகைகளை', 'O'), ('பரிசளிக்கக்கூடிய', 'O'), ('பிராண்ட்', 'O'), ('பெட்டியுடன்', 'O'), ('வழங்குகிறோம்', 'O'), (',